In [2]:
# !pip install fake_useragent

In [3]:
import requests
from fake_useragent import UserAgent #어뷰징 차단 방지용, 웹브라우저 랜덤으로 변경

In [27]:
url= 'http://www.zdnet.co.kr/newskey/?lstcode=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5'
headers = {
    "Accept":'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Cookie': 'PCID=15759513002304428115611; _ga=GA1.3.2146010160.1575951300; ASPSESSIONIDSSRRSDTD=LBDKPDEDDDDFGOONPNFGBODM; _gid=GA1.3.1317130600.1578015777; dable_uid=61491909.1549864149837; ASPSESSIONIDCQRQSARA=CNEACOEDHICOKOGJJAPIKIGN; _pk_ref.zdnet_web.cce1=%5B%22%22%2C%22%22%2C1578018081%2C%22https%3A%2F%2Fwww.google.com%2F%22%5D; _pk_ses.zdnet_web.cce1=1; OAX=3mvufV4OpVwADyGI; _pk_id.zdnet_web.cce1=712bb923a3e0d19c.1575951300.3.1578018240.1578018081.; _gat_gtag_UA_26701480_12=1',
    "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36',
    'Host': 'www.zdnet.co.kr',
    'Upgrade-Insecure-Requests': "1",
    "Connection": "keep-alive"
    
}
response= requests.get(url,headers=headers)
response

ConnectionError: HTTPConnectionPool(host='www.zdnet.co.kr', port=80): Max retries exceeded with url: /newskey/?lstcode=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000012FB9906D48>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))

### Selenium으로 크롤링

In [30]:
# !pip install selenium

  Using cached https://files.pythonhosted.org/packages/80/d6/4294f0b4bce4de0abf13e17190289f9d0613b0a44e5dd6a7f5ca98459853/selenium-3.141.0-py2.py3-none-any.whl


In [31]:
from selenium import webdriver

In [37]:
url= 'http://www.zdnet.co.kr/newskey/?lstcode=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5'

In [50]:
driver =  webdriver.Chrome()
driver.get(url)

In [51]:
#여러데이터 모으기
articles = driver.find_elements_by_css_selector("div.contentWrapper > div > div.left_cont > section > div:nth-child> div.assetText")
len(articles)

InvalidSelectorException: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=79.0.3945.88)


In [ ]:
/html/body/div[6]/div/div[1]/section/div[1]